# text mining (nlp) with python

**Author:** Ties de Kok ([Personal Website](http://www.tiesdekok.com))  
**Last updated:** 30 Oct 2017  
**Python version:** Python 3.5  
**License:** MIT License  

**Note:** Some features (like the ToC) will only work if you run the notebook or if you use nbviewer by clicking this link: 

# *Introduction*

This notebook contains code examples to get you started with Natural Language Processing (NLP) / Text Mining for Research and Data Science purposes.  

In the large scheme of things there are roughly 4 steps:  

1. Identify a data source  
2. Gather the data  
3. Process the data  
4. Analyze the data  

This notebook only discusses step 3 and 4. If you want to learn more about step 2 see my [Python tutorial](https://github.com/TiesdeKok/LearnPythonforResearch). 

## Note: companion slides

This notebook was designed to accompany a PhD course session on NLP techniques in Accounting Research.  
The slides of this session are publically availabe here: [Slides](http://www.tiesdekok.com/AccountingNLP_Slides/)

# *Elements / topics that are discussed in this notebook: *


<img style="float: left" src="https://i.imgur.com/c3aCZLA.png" width="50%" /> 

# *Table of Contents*  <a id='toc'></a>

* [Primer on NLP tools](#tool_primer)     
* [Process + Clean text](#proc_clean)   
    * [Normalization](#normalization)
        * [Deal with unwanted characters](#unwanted_char)
        * [Sentence segmentation](#sentence_seg)   
        * [Word tokenization](#word_token)
        * [Lemmatization & Stemming](#lem_and_stem) 
    * [Language modeling](#lang_model) 
        * [Part-of-Speech tagging](#pos_tagging) 
        * [Uni-Gram & N-Grams](#n_grams) 
        * [Stop words](#stop_words) 
* [Direct feature extraction](#feature_extract) 
    * [Feature search](#feature_search) 
        * [Entity recognition](#entity_recognition) 
        * [Pattern search](#pattern_search) 
    * [Text evaluation](#text_eval) 
        * [Language](#language) 
        * [Dictionary counting](#dict_counting) 
        * [Readability](#readability) 
* [Represent text numerically](#text_numerical) 
    * [Bag of Words](#bows) 
        * [TF-IDF](#tfidf) 
    * [Word Embeddings](#word_embed) 
        * [Word2Vec](#Word2Vec) 
* [Statistical models](#stat_models) 
    * ["Traditional" machine learning](#trad_ml) 
        * [Supervised](#trad_ml_supervised) 
            * [Naïve Bayes](#trad_ml_supervised_nb) 
            * [Support Vector Machines (SVM)](#trad_ml_supervised_svm) 
        * [Unsupervised](#trad_ml_unsupervised) 
            * [Latent Dirichilet Allocation (LDA)](#trad_ml_unsupervised_lda) 
            * [pyLDAvis](#trad_ml_unsupervised_pyLDAvis) 
* [Model Selection and Evaluation](#trad_ml_eval) 
* [Neural Networks](#nn_ml)

# <span style="text-decoration: underline;">Primer on NLP tools</span><a id='tool_primer'></a> [(to top)](#toc)

There are many tools available for NLP purposes.  
The code examples below are based on what I personally like to use, it is not intended to be a comprehsnive overview.  

Besides build-in Python functionality I will use / demonstrate the following packages:

**Standard NLP libraries**:
1. `Spacy` and the higher-level wrapper `Textacy` 
2. `NLTK` and the higher-level wrapper `TextBlob`

*Note: besides installing the above packages you also often have to download (model) data . Make sure to check the documentation!*

**Standard machine learning library**:

1. `scikit learn`

**Specific task libraries**:

There are many, just a couple of examples:

1. `pyLDAvis` for visualizing LDA)
2. `langdetect` for detecting languages
3. `fuzzywuzzy` for fuzzy text matching
4. `textstat` to calculate readability statistics
5. `Gensim` for topic modelling

# <span style="text-decoration: underline;">Get some example data</span><a id='example_data'></a> [(to top)](#toc)

There are many example datasets available to play around with, see for example this great repository:  
https://archive.ics.uci.edu/ml/datasets.html?format=&task=&att=&area=&numAtt=&numIns=&type=text&sort=nameUp&view=table

The data that I will use for most of the examples is the "Reuter_50_50 Data Set" that is used for author identification experiments. 

See the details here: https://archive.ics.uci.edu/ml/datasets/Reuter_50_50  

### Download and load the data

Can't follow what I am doing here? Please see my [Python tutorial](https://github.com/TiesdeKok/LearnPythonforResearch) (although the `zipfile` and `io` operations are not very relevant).

In [1]:
import requests, zipfile, io, os

*Download and extract the zip file with the data *

In [2]:
if not os.path.exists('C50test'):
    r = requests.get("https://archive.ics.uci.edu/ml/machine-learning-databases/00217/C50.zip")
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()

*Load the data into memory*

In [3]:
folder_dict = {'test' : 'C50test', 'train' : 'C50train'}
text_dict = {'test' : {}, 'train' : {}}

In [4]:
for label, folder in folder_dict.items():
    authors = os.listdir(folder)
    for author in authors:
        text_files = os.listdir(os.path.join(folder, author))
        for file in text_files:
            with open(os.path.join(folder, author, file), 'r') as text_file:
                text_dict[label].setdefault(author, []).append(' '.join(text_file.readlines()))

*Note: the text comes pre-split per sentence, for the sake of example I undo this through `' '.join(text_file.readlines()`*

In [5]:
text_dict['test']['TimFarrand'][0]

'Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain\'s Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.\n Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers Commission which is due to report before March 24, 1997. The shares fell 6p to 781p on the news.\n "The stock is probably dead in the water until March," said John Wakley, analyst at Lehman Brothers.  \n Dermott Carr, an analyst at Nikko said, "the market is going to hang onto them for the moment but until we get a decision they will be held back."\n Whatever the MMC decides many analysts expect Lang to defer a decision until after the next general election which will be called by May 22.\n "They will probably try to defer the decision until after the election. I don\'t think they want the negative PR of having a large number of people fired," said Wakley.  \n If the deal does not go throu

# <span style="text-decoration: underline;">Process + Clean text</span><a id='proc_clean'></a> [(to top)](#toc)

## Convert the text into a NLP representation

We can use the text directly, but if want to use packages like `spacy` and `textblob` we first have to convert the text into a corresponding object.  

### Spacy

In [6]:
from spacy.en import English
parser = English()

Convert all text in the "test" sample to a `spacy` `doc` object using `parser()`:

In [7]:
spacy_text = {}
for author, text_list in text_dict['test'].items():
    spacy_text[author] = [parser(text) for text in text_list]

In [8]:
type(spacy_text['TimFarrand'][0])

spacy.tokens.doc.Doc

### NLTK

In [9]:
import nltk

We can apply basic `nltk` operations directly to the text so we don't need to convert first. 

### TextBlob

In [10]:
from textblob import TextBlob

Convert all text in the "test" sample to a `TextBlob` object using `TextBlob()`:

In [11]:
textblob_text = {}
for author, text_list in text_dict['test'].items():
    textblob_text[author] = [TextBlob(text) for text in text_list]

In [12]:
type(textblob_text['TimFarrand'][0])

textblob.blob.TextBlob

## <span style="text-decoration: underline;">Normalization</span><a id='normalization'></a> [(to top)](#toc)

**Text normalization** describes the task of transforming the text into a different (more comparable) form.  

This can imply many things, I will show a couple of things below:

### <span style="text-decoration: underline;">Deal with unwanted characters</span><a id='unwanted_char'></a> [(to top)](#toc)

You will often notice that there are characters that you don't want in your text.  

Let's look at this sentence for example:

> "Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain\'s Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.\n Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers"

You notice that there are some `\` and `\n` in there. These are used to define how a string should be displayed, if we print this text we get:  

In [13]:
text_dict['test']['TimFarrand'][0][:298]

"Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.\n Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers"

In [14]:
print(text_dict['test']['TimFarrand'][0][:298])

Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.
 Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers


If we want to analyze text we often don't care about the visual representation. They might actually cause problems!  

** So how do we remove them? **

In many cases it is sufficient to simply use the `.replace()` function:

In [15]:
text_dict['test']['TimFarrand'][0][:298].replace('\n', '').replace('\\', '')

"Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts. Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers"

Sometimes, however, the problem arrises because of encoding / decoding problems.  

In those cases you can usually do something like:  

In [16]:
problem_sentence = 'This is some \\u03c0 text that has to be cleaned\\u2026! it\\u0027s annoying!'
print(problem_sentence.encode().decode('unicode_escape').encode('ascii','ignore'))

b"This is some  text that has to be cleaned! it's annoying!"


### <span style="text-decoration: underline;">Sentence segmentation</span><a id='sentence_seg'></a> [(to top)](#toc)

Sentence segmentation means the task of splitting up the piece of text by sentence.  

You could do this by splitting on the `.` symbol, but dots are used in many other cases as well so it is not very robust:

In [17]:
text_dict['test']['TimFarrand'][0][:550].split('.')

["Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts",
 '\n Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers Commission which is due to report before March 24, 1997',
 ' The shares fell 6p to 781p on the news',
 '\n "The stock is probably dead in the water until March," said John Wakley, analyst at Lehman Brothers',
 '  \n Dermott Carr, an analyst at Nikko said, "the mark']

It is better to use a more sophisticated implementation such as the one by `Spacy`:

In [18]:
example_paragraph = spacy_text['TimFarrand'][0]

In [19]:
sentence_list = [s for s in example_paragraph.sents]
sentence_list[:5]

[Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.
  ,
 Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers Commission which is due to report before March 24, 1997.,
 The shares fell 6p to 781p on the news.
  ,
 "The stock is probably dead in the water until March," said John Wakley, analyst at Lehman Brothers.  
  ,
 Dermott Carr, an analyst at Nikko said, "the market is going to hang onto them for the moment but until we get a decision they will be held back."
  ]

Notice that the returned object is still a `spacy` object:

In [20]:
type(sentence_list[0])

spacy.tokens.span.Span

Apply to all texts (for use later on):

In [21]:
spacy_sentences = {}
for author, text_list in spacy_text.items():
    spacy_sentences[author] = [list(text.sents) for text in text_list]

In [22]:
spacy_sentences['TimFarrand'][0][:3]

[Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.
  ,
 Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers Commission which is due to report before March 24, 1997.,
 The shares fell 6p to 781p on the news.
  ]

### <span style="text-decoration: underline;">Word tokenization</span><a id='word_token'></a> [(to top)](#toc)

Word tokenization means to split the sentence (or text) up into words.

In [23]:
example_sentence = spacy_sentences['TimFarrand'][0][0]
example_sentence

Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.
 

A word is called a `token` in this context (hence `tokenization`), using `spacy`:

In [24]:
token_list = [token for token in example_sentence]
token_list[0:15]

[Shares,
 in,
 brewing,
 -,
 to,
 -,
 leisure,
 group,
 Bass,
 Plc,
 are,
 likely,
 to,
 be,
 held]

### <span style="text-decoration: underline;">Lemmatization & Stemming</span><a id='lem_and_stem'></a> [(to top)](#toc)

In some cases you want to convert a word (i.e. token) into a more general representation.  

For example: convert "car", "cars", "car's", "cars'" all into the word `car`.

This is generally done through lemmatization / stemming (different approaches trying to achieve a similar goal).  

**Spacy**

Space offers build-in functionality for lemmatization:

In [25]:
lemmatized = [token.lemma_ for token in example_sentence]
lemmatized[0:15]

['share',
 'in',
 'brewing',
 '-',
 'to',
 '-',
 'leisure',
 'group',
 'bass',
 'plc',
 'be',
 'likely',
 'to',
 'be',
 'hold']

**NLTK**

Using the NLTK libary we can also use the more aggressive Porter Stemmer

In [26]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [27]:
stemmed = [stemmer.stem(token.text) for token in example_sentence]
stemmed[0:15]

['share',
 'in',
 'brew',
 '-',
 'to',
 '-',
 'leisur',
 'group',
 'bass',
 'plc',
 'are',
 'like',
 'to',
 'be',
 'held']

**Compare**:

In [28]:
for original, lemma, stem in zip(token_list[:15], lemmatized[:15], stemmed[:15]):
    print(original, ' | ', lemma, ' | ', stem)

Shares  |  share  |  share
in  |  in  |  in
brewing  |  brewing  |  brew
-  |  -  |  -
to  |  to  |  to
-  |  -  |  -
leisure  |  leisure  |  leisur
group  |  group  |  group
Bass  |  bass  |  bass
Plc  |  plc  |  plc
are  |  be  |  are
likely  |  likely  |  like
to  |  to  |  to
be  |  be  |  be
held  |  hold  |  held


In my experience it is usually best to use lemmatization instead of a stemmer. 

## <span style="text-decoration: underline;">Language modeling</span><a id='lang_model'></a> [(to top)](#toc)

Text is inherently structured in complex ways, we can often use some of this underlying structure. 

### <span style="text-decoration: underline;">Part-of-Speech tagging</span><a id='pos_tagging'></a> [(to top)](#toc)

Part of speech tagging refers to the identification of words as nouns, verbs, adjectives, etc. 

Using `Spacy`:

In [29]:
pos_list = [(token, token.pos_) for token in example_sentence]
pos_list[0:10]

[(Shares, 'NOUN'),
 (in, 'ADP'),
 (brewing, 'NOUN'),
 (-, 'PUNCT'),
 (to, 'ADP'),
 (-, 'PUNCT'),
 (leisure, 'NOUN'),
 (group, 'NOUN'),
 (Bass, 'PROPN'),
 (Plc, 'PROPN')]

### <span style="text-decoration: underline;">Uni-Gram & N-Grams</span><a id='n_grams'></a> [(to top)](#toc)

Obviously a sentence is not a random collection of words, the sequence of words has information value.  

A simple way to incorporate some of this sequence is by using what is called `n-grams`.  
An `n-gram` is nothing more than a a combination of `N` words into one token (a uni-gram token is just one word).  

So we can convert `"Sentence about flying cars"` into a list of bigrams:

> Sentence-about, about-flying, flying-cars  

See my slide on N-Grams for a more comprehensive example: [click here](http://www.tiesdekok.com/AccountingNLP_Slides/#14)

Using `NLTK`:

In [30]:
bigram_list = ['-'.join(x) for x in nltk.bigrams([token.text for token in example_sentence])]
bigram_list[10:15]

['are-likely', 'likely-to', 'to-be', 'be-held', 'held-back']

### <span style="text-decoration: underline;">Stop words</span><a id='stop_words'></a> [(to top)](#toc)

Depending on what you are trying to do it is possible that there are many words that don't add any information value to the sentence.  

The primary example are stop words.  

Sometimes you can improve the accuracy of your model by removing stop words.

Using `Spacy`:

In [31]:
no_stop_words = [token for token in example_sentence if not token.is_stop]

In [32]:
no_stop_words[:10]

[Shares, brewing, -, -, leisure, group, Bass, Plc, likely, held]

In [33]:
token_list[:10]

[Shares, in, brewing, -, to, -, leisure, group, Bass, Plc]

*Note* we can also remove punctuation in the same way:

In [34]:
[token for token in example_sentence if not token.is_punct][:5]

[Shares, in, brewing, to, leisure]

## Wrap everything into one function

Below I will primarily use `SpaCy` directly. However, I also recommend to check out the high-level wrapper `Textacy`.

See their GitHub page for details: https://github.com/chartbeat-labs/textacy

### Quick `Textacy` example

In [35]:
import textacy

In [36]:
example_text = text_dict['test']['TimFarrand'][0]

In [37]:
cleaned_text = textacy.preprocess_text(example_text, lowercase=True, fix_unicode=True, no_punct=True)

** Basic SpaCy text processing function **

1. Split into sentences
2. Apply lemmatizer and remove top words
3. Clean up the sentence using `textacy`

In [38]:
def process_text_custom(text):
    sentences = list(parser(text).sents)
    lemmatized_sentences = []
    for sentence in sentences:
        lemmatized_sentences.append([token.lemma_ for token in sentence if not token.is_stop | token.is_punct | token.is_space])
    return [parser(' '.join(sentence)) for sentence in lemmatized_sentences]

In [39]:
%%time
spacy_text_clean = {}
for author, text_list in text_dict['test'].items():
    lst = []
    for text in text_list:
        lst.append(process_text_custom(text))
    spacy_text_clean[author] = lst

CPU times: user 1min 47s, sys: 108 ms, total: 1min 47s
Wall time: 1min 47s


Note that there are quite a lot of sentences (~52K) so this takes a bit of time (~ 2 minutes).

In [40]:
count = 0
for author, texts in spacy_text_clean.items():
    for text in texts:
        count += len(text)
print('Number of sentences:', count)

Number of sentences: 53538


Result

In [41]:
spacy_text_clean['TimFarrand'][0][:3]

[share brewing leisure group bass plc likely hold britain 's trade industry secretary ian lang decide allow propose merge brewer carlsberg tetley say analyst,
 earlier lang announce bass deal refer monoplies mergers commission report march 24 1997,
 share fall 6p 781p news]

# <span style="text-decoration: underline;">Direct feature extraction</span><a id='feature_extract'></a> [(to top)](#toc)

We now have pre-processed our text into something that we can use for direct feature extraction or to convert it to a numerical representation. 

## <span style="text-decoration: underline;">Feature search</span><a id='feature_search'></a> [(to top)](#toc)

### <span style="text-decoration: underline;">Entity recognition</span><a id='entity_recognition'></a> [(to top)](#toc)

It is often useful / relevant to extract entities that are mentioned in a piece of text.   

SpaCy is quite powerful in extracting entities, however, it doesn't work very well on lowercase text.  

Given that "token.lemma\_" removes capitalization I will use `spacy_sentences` for this example.

In [42]:
example_sentence = spacy_sentences['TimFarrand'][0][3]
example_sentence

"The stock is probably dead in the water until March," said John Wakley, analyst at Lehman Brothers.  
 

In [43]:
[(i, i.label_) for i in parser(example_sentence.text).ents]

[(March, 'DATE'), (John Wakley, 'PERSON'), (Lehman Brothers, 'ORG')]

In [44]:
example_sentence = spacy_sentences['TimFarrand'][4][0]
example_sentence

British pub-to-hotel group Greenalls Plc on Thursday reported a 48 percent rise in profits before exceptional items to 148.7 million pounds ($246.4 million), driven by its acquisition of brewer Boddington in November 1995.
 "

In [45]:
[(i, i.label_) for i in parser(example_sentence.text).ents]

[(British, 'NORP'),
 (Greenalls Plc, 'ORG'),
 (Thursday, 'DATE'),
 (48 percent, 'PERCENT'),
 (148.7 million pounds, 'QUANTITY'),
 ($246.4 million, 'MONEY'),
 (Boddington, 'ORG'),
 (November 1995, 'DATE')]

### <span style="text-decoration: underline;">Pattern search</span><a id='pattern_search'></a> [(to top)](#toc)

Using the build-in `re` (regular expression) library you can pattern match nearly anything you want.  

I will not go into details about regular expressions but see here for a tutorial:  
https://regexone.com/references/python  

In [46]:
import re

**TIP**: Use [Pythex.org](https://pythex.org/) to try out your regular expression

Example on Pythex: <a href="https://pythex.org/?regex=IDNUMBER: (\d\d\d-\w\w)&test_string=Ties de Kok (IDNUMBER: 123-AZ). Rest of Text." target='_blank'>click here</a>

**Example 1:**  

In [47]:
string_1 = 'Ties de Kok (#IDNUMBER: 123-AZ). Rest of text...'
string_2 = 'Philip Joos (#IDNUMBER: 663-BY). Rest of text...'

In [48]:
pattern = r'#IDNUMBER: (\d\d\d-\w\w)'

In [49]:
print(re.findall(pattern, string_1)[0])
print(re.findall(pattern, string_2)[0])

123-AZ
663-BY


### Example 2:

If a sentence contains the word 'million' return True, otherwise return False

In [50]:
for sen in spacy_text_clean['TimFarrand'][2]:
    TERM = 'million'
    contains = True if re.search('million', sen.text) else False
    if contains:
        print(sen)

analyst forecast pretax profit range 218 232 million stg restructure cost 206 million time
restructure cost 35 million anticipate bulk 25 million stem closure small production plant france
cadbury 's u.s. drink business turn 112 million stg trading profit 59 million half 1995 entirely contribution dr pepper
campbell estimate uk beverage contribute 47 million stg operating profit 50 million time
broadly analyst expect pretty flat performance group 's confectionery business consensus forecast 110 million stg operate profit
average analyst calculate beverage chip trading profit 150 million
sale 51 percent stake coca cola amp schweppes beverages ccsb operation coca cola enterprises june 620 million stg analyst want clear statement strategy company
far analyst company say shareholder expect return investment emerge market large far 75 million russian plant
cadbury announce investment 20 million stg build new plant wrocoaw poland 1993 joint venture china cost 20 million
net debt 1.34 billion

## <span style="text-decoration: underline;">Text evaluation</span><a id='text_eval'></a> [(to top)](#toc)

Besides feature search there are also many ways to analyze the text as a whole.  

Let's, for example, evaluate the following paragraph:

In [51]:
example_paragraph = ' '.join([x.text for x in spacy_text_clean['TimFarrand'][2]])
example_paragraph[:500]

"soft drink confectionery group cadbury schweppes plc expect report solid percent rise half profit wednesday face question performance 7up soft drink u.s. main question success relaunch 7up brand say mark duffy food manufacturing analyst sbc warburg competitor sprite own coca cola see agressive marketing push rank fast grow brand u.s. cadbury 's dr pepper analyst forecast pretax profit range 218 232 million stg restructure cost 206 million time dividend 5.1 penny expect 4.9p restructure cost 35 m"

### <span style="text-decoration: underline;">Language</span><a id='language'></a> [(to top)](#toc)

Using the `langdetect` package it is easy to detect the language of a piece of text

In [52]:
from langdetect import detect

In [53]:
detect(example_paragraph)

'en'

### <span style="text-decoration: underline;">Readability</span><a id='readability'></a> [(to top)](#toc)

Using the `textstat` package we can compute various readability metrics

https://github.com/shivam5992/textstat

In [54]:
from textstat.textstat import textstat

In [55]:
print(textstat.flesch_reading_ease(example_paragraph))
print(textstat.smog_index(example_paragraph))
print(textstat.flesch_kincaid_grade(example_paragraph))
print(textstat.coleman_liau_index(example_paragraph))
print(textstat.automated_readability_index(example_paragraph))
print(textstat.dale_chall_readability_score(example_paragraph))
print(textstat.difficult_words(example_paragraph))
print(textstat.linsear_write_formula(example_paragraph))
print(textstat.gunning_fog(example_paragraph))
print(textstat.text_standard(example_paragraph))

23.02
12.9
19.8
16.72
25.4
10.4
120
8.333333333333334
29.942741116751268
12th and 13th grade


## Text similarity

In [56]:
from fuzzywuzzy import fuzz

In [57]:
fuzz.ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear")

91

### <span style="text-decoration: underline;">Term (dictionary) counting</span><a id='dict_counting'></a> [(to top)](#toc)

One of the most common techniques that researchers currently use (at least in Accounting research) are simple metrics based on counting words in a dictionary.  
This technique is, for example, very prevalent in sentiment analysis (counting positive and negative words).  

In essence this technique is very simple to program:

### Example 1:

In [58]:
word_dictionary = ['soft', 'first', 'most', 'be']

In [59]:
for word in word_dictionary:
    print(word, example_paragraph.count(word))

soft 3
first 0
most 0
be 8


### Example 2:

In [60]:
pos = ['great', 'increase']
neg = ['bad', 'decrease']

In [61]:
sentence = '''According to Trump everything is great, great, 
and great even though his popularity is seeing a decrease.'''

In [62]:
pos_count = 0
for word in pos:
    pos_count += sentence.lower().count(word)
print(pos_count)

3


In [63]:
neg_count = 0
for word in neg:
    neg_count += sentence.lower().count(word)
print(neg_count)

1


In [64]:
pos_count / (neg_count + pos_count)

0.75

Getting the total number of words is also easy:

In [65]:
len(parser(example_paragraph))

401

#### Example 3:

We can also save the count per word

In [66]:
pos_count_dict = {}
for word in pos:
    pos_count_dict[word] = sentence.lower().count(word)

In [67]:
pos_count_dict

{'great': 3, 'increase': 0}

# <span style="text-decoration: underline;">Represent text numerically</span><a id='text_numerical'></a> [(to top)](#toc)

## <span style="text-decoration: underline;">Bag of Words</span><a id='bows'></a> [(to top)](#toc)

Sklearn includes the `CountVectorizer` and `TfidfVectorizer` function.  

For details, see the documentation:  
[TF](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)  
[TFIDF](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer)

Note 1: these functions also already includes a lot of preprocessing options (e.g. ngrames, remove stop words, accent stripper).

Note 2: example based on the following website [click here](http://ethen8181.github.io/machine-learning/clustering_old/tf_idf/tf_idf.html)

In [68]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

### Simple example:

In [69]:
doc_1 = "The sky is blue."
doc_2 = "The sun is bright today."
doc_3 = "The sun in the sky is bright."
doc_4 = "We can see the shining sun, the bright sun."

Calculate term frequency:

In [70]:
vectorizer = CountVectorizer(stop_words='english')
tf = vectorizer.fit_transform([doc_1, doc_2, doc_3, doc_4])

In [71]:
print(vectorizer.get_feature_names())
for doc_tf_vector in tf.toarray():
    print(doc_tf_vector)

['blue', 'bright', 'shining', 'sky', 'sun', 'today']
[1 0 0 1 0 0]
[0 1 0 0 1 1]
[0 1 0 1 1 0]
[0 1 1 0 2 0]


### <span style="text-decoration: underline;">TF-IDF</span><a id='tfidf'></a> [(to top)](#toc)

In [72]:
transformer = TfidfVectorizer(stop_words='english')
tfidf = transformer.fit_transform([doc_1, doc_2, doc_3, doc_4])

In [73]:
for doc_vector in tfidf.toarray():
    print(doc_vector)

[ 0.78528828  0.          0.          0.6191303   0.          0.        ]
[ 0.          0.47380449  0.          0.          0.47380449  0.74230628]
[ 0.          0.53256952  0.          0.65782931  0.53256952  0.        ]
[ 0.          0.36626037  0.57381765  0.          0.73252075  0.        ]


### More elaborate example:

In [74]:
clean_paragraphs = []
for author, value in spacy_text_clean.items():
    for article in value:
        clean_paragraphs.append(' '.join([x.text for x in article]))

In [75]:
len(clean_paragraphs)

2500

In [76]:
transformer = TfidfVectorizer(stop_words='english')
tfidf_large = transformer.fit_transform(clean_paragraphs)

In [77]:
print('Number of vectors:', len(tfidf_large.toarray()))
print('Number of words in dictionary:', len(tfidf_large.toarray()[0]))

Number of vectors: 2500
Number of words in dictionary: 24141


In [78]:
tfidf_large

<2500x24141 sparse matrix of type '<class 'numpy.float64'>'
	with 444634 stored elements in Compressed Sparse Row format>

## <span style="text-decoration: underline;">Word Embeddings</span><a id='word_embed'></a> [(to top)](#toc)

### <span style="text-decoration: underline;">Word2Vec</span><a id='Word2Vec'></a> [(to top)](#toc)

Simple example below is from:  https://medium.com/@mishra.thedeepak/word2vec-in-minutes-gensim-nlp-python-6940f4e00980

In [79]:
import gensim
from nltk.corpus import brown

In [80]:
sentences = brown.sents()
model = gensim.models.Word2Vec(sentences, min_count=1)

Save model

In [81]:
model.save('brown_model')

Load model

In [82]:
model = gensim.models.Word2Vec.load('brown_model')

Find words most similar to 'mother':

In [83]:
print(model.most_similar("mother"))

[('father', 0.9751995801925659), ('husband', 0.9580737352371216), ('wife', 0.9531918168067932), ('son', 0.9327206611633301), ('voice', 0.9201934337615967), ('boy', 0.9148358106613159), ('friend', 0.9091534614562988), ('ache', 0.8969202041625977), ('parents', 0.8916678428649902), ('maid', 0.8907431960105896)]


Find the odd one out:

In [84]:
print(model.doesnt_match("breakfast cereal dinner lunch".split()))

cereal


In [85]:
print(model.doesnt_match("pizza pasta garden fries".split()))

garden


Retrieve vector representation of the word "human"

In [86]:
model['human']

array([ -1.24233282e+00,   1.73863158e-01,  -6.12607360e-01,
        -5.31224608e-01,   2.60317028e-01,  -3.56246114e-01,
        -5.06218493e-01,  -1.15960002e-01,   3.04902345e-01,
        -1.21463396e-01,   4.93332326e-01,  -9.75235820e-01,
         3.44474047e-01,   9.77401040e-04,   5.05028106e-02,
        -3.87262404e-01,   4.93362129e-01,   7.00087488e-01,
        -6.27336025e-01,  -4.63026613e-01,   2.79739406e-02,
         1.45691419e+00,   4.07162786e-01,  -4.19379741e-01,
        -8.41612220e-01,   8.46711546e-02,   3.03834379e-01,
        -5.89724183e-01,  -5.50288737e-01,   1.14675418e-01,
        -9.28169414e-02,  -5.00818849e-01,   1.74140222e-02,
         2.45587930e-01,   9.37732458e-02,  -6.30766377e-02,
         3.79322648e-01,  -9.30945396e-01,   1.81099728e-01,
         4.46529061e-01,   5.09826422e-01,  -4.00113940e-01,
        -3.06686193e-01,   5.83700202e-02,  -1.30845475e+00,
        -8.19562197e-01,  -1.43999264e-01,  -1.79302439e-01,
        -9.88642037e-01,

# <span style="text-decoration: underline;">Statistical models</span><a id='stat_models'></a> [(to top)](#toc)

## <span style="text-decoration: underline;">"Traditional" machine learning</span><a id='trad_ml'></a> [(to top)](#toc)

The library to use for machine learning is scikit-learn (["sklearn"](http://scikit-learn.org/stable/index.html)).

## <span>Supervised</span><a id='trad_ml_supervised'></a> [(to top)](#toc)

In [87]:
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.externals import joblib

In [88]:
import pandas as pd
import numpy as np

### Convert the data into a pandas dataframe (so that we can input it easier)

In [89]:
article_list = []
for author, value in spacy_text_clean.items():
    for article in value:
        article_list.append((author, ' '.join([x.text for x in article])))

In [90]:
article_df = pd.DataFrame(article_list, columns=['author', 'text'])

In [91]:
article_df.sample(5)

,author,text
1738,AlanCrosby,czech consumer inflation continue gradual down...
710,LydiaZajc,toronto stock end mixed light dealing monday w...
419,KirstinRidley,telecom analyst london amuse monday portugal t...
2092,EricAuchard,domestic trouble at&amp;t corp contend new thr...
2229,NickLouth,regional bell telephone company report strong ...


### Split the sample into a training and test sample

In [92]:
X_train, X_test, y_train, y_test = train_test_split(article_df.text, article_df.author, test_size=0.20, random_state=3561)

In [93]:
print(len(X_train), len(X_test))

2000 500


### Train and evaluate function

Simple function to train (i.e. fit) and evaluate the model

In [94]:
def train_and_evaluate(clf, X_train, X_test, y_train, y_test):
    
    clf.fit(X_train, y_train)
    
    print("Accuracy on training set:")
    print(clf.score(X_train, y_train))
    print("Accuracy on testing set:")
    print(clf.score(X_test, y_test))
    
    y_pred = clf.predict(X_test)
    
    print("Classification Report:")
    print(metrics.classification_report(y_test, y_pred))

### <span>Naïve Bayes estimator</span><a id='trad_ml_supervised_nb'></a> [(to top)](#toc)

In [95]:
from sklearn.naive_bayes import MultinomialNB

Define pipeline

In [96]:
clf = Pipeline([
    ('vect', TfidfVectorizer(strip_accents='unicode',
                             lowercase = True,
                            max_features = 1500,
                            stop_words='english'
                            )),
        
    ('clf', MultinomialNB(alpha = 1,
                          fit_prior = True
                          )
    ),
])

Train and show evaluation stats

In [97]:
train_and_evaluate(clf, X_train, X_test, y_train, y_test)

Accuracy on training set:
0.8475
Accuracy on testing set:
0.7
Classification Report:
                   precision    recall  f1-score   support

    AaronPressman       0.89      0.73      0.80        11
       AlanCrosby       0.53      0.90      0.67        10
   AlexanderSmith       1.00      0.78      0.88         9
  BenjaminKangLim       0.60      0.60      0.60        10
    BernardHickey       0.88      0.78      0.82         9
      BradDorfman       0.57      0.80      0.67         5
 DarrenSchuettler       0.75      0.60      0.67        10
      DavidLawder       0.58      0.78      0.67         9
    EdnaFernandes       1.00      0.30      0.46        10
      EricAuchard       0.67      0.29      0.40        14
   FumikoFujisaki       1.00      1.00      1.00        13
   GrahamEarnshaw       0.71      1.00      0.83        10
 HeatherScoffield       0.67      0.60      0.63        10
       JanLopatka       0.54      0.70      0.61        10
    JaneMacartney       0.62 

Save results

In [98]:
joblib.dump(clf, 'naive_bayes_results.pkl')

['naive_bayes_results.pkl']

Predict out of sample:

In [99]:
example_y, example_X = y_train[33], X_train[33]

In [100]:
print('Actual author:', example_y)
print('Predicted author:', clf.predict([example_X])[0])

Actual author: JoeOrtiz
Predicted author: AlexanderSmith


### <span>Support Vector Machines (SVM)</span><a id='trad_ml_supervised_svm'></a> [(to top)](#toc)

In [101]:
from sklearn.svm import SVC

Define pipeline

In [102]:
clf_svm = Pipeline([
    ('vect', TfidfVectorizer(strip_accents='unicode',
                             lowercase = True,
                            max_features = 1500,
                            stop_words='english'
                            )),
        
    ('clf', SVC(kernel='rbf' ,
                C=10, gamma=0.3)
    ),
])

*Note:* The SVC estimator is very sensitive to the hyperparameters!

Train and show evaluation stats

In [103]:
train_and_evaluate(clf_svm, X_train, X_test, y_train, y_test)

Accuracy on training set:
0.998
Accuracy on testing set:
0.814
Classification Report:
                   precision    recall  f1-score   support

    AaronPressman       0.91      0.91      0.91        11
       AlanCrosby       0.83      1.00      0.91        10
   AlexanderSmith       1.00      0.67      0.80         9
  BenjaminKangLim       0.58      0.70      0.64        10
    BernardHickey       1.00      0.89      0.94         9
      BradDorfman       0.80      0.80      0.80         5
 DarrenSchuettler       0.90      0.90      0.90        10
      DavidLawder       0.46      0.67      0.55         9
    EdnaFernandes       1.00      0.90      0.95        10
      EricAuchard       0.73      0.57      0.64        14
   FumikoFujisaki       1.00      1.00      1.00        13
   GrahamEarnshaw       0.91      1.00      0.95        10
 HeatherScoffield       0.90      0.90      0.90        10
       JanLopatka       0.78      0.70      0.74        10
    JaneMacartney       0.60

Save results

In [104]:
joblib.dump(clf_svm, 'svm_results.pkl')

['svm_results.pkl']

Predict out of sample:

In [105]:
example_y, example_X = y_train[33], X_train[33]

In [106]:
print('Actual author:', example_y)
print('Predicted author:', clf_svm.predict([example_X])[0])

Actual author: JoeOrtiz
Predicted author: JoeOrtiz


## <span>Model Selection and Evaluation</span><a id='trad_ml_eval'></a> [(to top)](#toc)

Both the `TfidfVectorizer` and `SVC()` estimator take a lot of hyperparameters.  

It can be difficult to figure out what the best parameters are.

We can use `GridSearchCV` to help figure this out.

In [107]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

First we define the options that should be tried out:

In [108]:
print('lala')

lala


In [109]:
print('test')

test


In [110]:
clf_search = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', SVC())
])
parameters = { 'vect__stop_words': ['english'],
                'vect__strip_accents': ['unicode'],
              'vect__max_features' : [1500],
              'vect__ngram_range': [(1,1), (2,2) ],
             'clf__gamma' : [0.2, 0.3, 0.4], 
             'clf__C' : [8, 10, 12],
              'clf__kernel' : ['rbf']
             }

Run everything:

In [111]:
grid = GridSearchCV(clf_search, param_grid=parameters, scoring=make_scorer(f1_score, average='micro'), n_jobs=6)
grid.fit(X_train, y_train)    

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params=None, iid=True, n_jobs=6,
       param_grid={'vect__stop_words': ['english'], 'clf__C': [8, 10, 12], 'clf__gamma': [0.2, 0.3, 0.4], 'clf__kernel': ['rbf'], 'vect__strip_accents': ['unicode'], 'vect__max_features': [1500], 'vect__ngram_range': [(1, 1), (2, 2)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(f1_score, average=micro), verbose=0)

*Note:* if you are on a powerful unix system you can set n_jobs to the number of available threads to speed up the calculation

In [112]:
print("The best parameters are %s with a score of %0.2f" % (grid.best_params_, grid.best_score_))
y_true, y_pred = y_test, grid.predict(X_test)
print(metrics.classification_report(y_true, y_pred))

The best parameters are {'vect__stop_words': 'english', 'clf__C': 10, 'clf__gamma': 0.3, 'clf__kernel': 'rbf', 'vect__strip_accents': 'unicode', 'vect__max_features': 1500, 'vect__ngram_range': (1, 1)} with a score of 0.78
                   precision    recall  f1-score   support

    AaronPressman       0.91      0.91      0.91        11
       AlanCrosby       0.83      1.00      0.91        10
   AlexanderSmith       1.00      0.67      0.80         9
  BenjaminKangLim       0.58      0.70      0.64        10
    BernardHickey       1.00      0.89      0.94         9
      BradDorfman       0.80      0.80      0.80         5
 DarrenSchuettler       0.90      0.90      0.90        10
      DavidLawder       0.46      0.67      0.55         9
    EdnaFernandes       1.00      0.90      0.95        10
      EricAuchard       0.73      0.57      0.64        14
   FumikoFujisaki       1.00      1.00      1.00        13
   GrahamEarnshaw       0.91      1.00      0.95        10
 HeatherS

## <span>Unsupervised</span><a id='trad_ml_unsupervised'></a> [(to top)](#toc)

### <span>Latent Dirichilet Allocation (LDA)</span><a id='trad_ml_unsupervised_lda'></a> [(to top)](#toc)

In [113]:
from sklearn.decomposition import LatentDirichletAllocation

Vectorizer (using countvectorizer for the sake of example)

In [114]:
vectorizer = CountVectorizer(strip_accents='unicode',
                             lowercase = True,
                            max_features = 1500,
                            stop_words='english', max_df=0.8)
tf_large = vectorizer.fit_transform(clean_paragraphs)

Run the LDA model

In [115]:
n_topics = 10
n_top_words = 25

In [116]:
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=10,
                                learning_method='online',
                                n_jobs=1)
lda_fitted = lda.fit_transform(tf_large)

Visualize top words

In [117]:
def save_top_words(model, feature_names, n_top_words):
    out_list = []
    for topic_idx, topic in enumerate(model.components_):
        out_list.append((topic_idx+1, " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])))
    out_df = pd.DataFrame(out_list, columns=['topic_id', 'top_words'])
    return out_df

In [118]:
result_df = save_top_words(lda, vectorizer.get_feature_names(), n_top_words)

In [119]:
result_df

,topic_id,top_words
0,1,kong hong china chinese deng party beijing peo...
1,2,million percent share analyst sale quarter pro...
2,3,stock gold gm bre canada toronto share busang ...
3,4,new market company plan group business add 000...
4,5,bank thomson banking financial nomura firm com...
5,6,company corp service computer new apple softwa...
6,7,market percent rate china bank crown billion a...
7,8,tonne 000 price export million crop source tra...
8,9,china official beijing chinese people air boei...
9,10,government ford car company czech strike new p...


### <span>pyLDAvis</span><a id='trad_ml_unsupervised_pyLDAvis'></a> [(to top)](#toc)

In [120]:
%matplotlib inline
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [121]:
pyLDAvis.sklearn.prepare(lda, tf_large, vectorizer, n_jobs=1)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      21.924676        1       1 -0.160148 -0.007949
5      14.437502        1       2 -0.015733 -0.123081
3      12.920510        1       3  0.012238 -0.053868
0       9.698103        1       4  0.226205  0.033388
9       8.993744        1       5  0.085544 -0.023278
4       8.146445        1       6 -0.034196 -0.122927
8       7.756998        1       7  0.198601  0.063036
6       7.555307        1       8 -0.078050  0.105785
7       4.549334        1       9 -0.121892  0.221830
2       4.017381        1      10 -0.112568 -0.092937, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
775   Default  2490.000000         kong  2490.000000  30.0000  30.0000
683   Default  2479.000000         hong  2479.000000  29.0000  29.0000
195   Default  2676.000000         bank  2676.000000  28.0000  28.0000
298   Default  3429.000000        china  3429.000000  27.0000  27.0000
997   Default  5566.000000      percent  5566.000000  26.0000  26.0000
1241  Default  3436.000000        share  3436.000000  25.0000  25.0000
1387  Default   975.000000        tonne   975.000000  24.0000  24.0000
1315  Default  2220.000000        stock  2220.000000  23.0000  23.0000
852   Default  4203.000000       market  4203.000000  22.0000  22.0000
1055  Default  2038.000000        price  2038.000000  21.0000  21.0000
882   Default  4990.000000      million  4990.000000  20.0000  20.0000
0     Default  1698.000000          000  1698.000000  19.0000  19.0000
1103  Default  1700.000000      quarter  1700.000000  18.0000  18.0000
342   Default  5385.000000      company  5385.000000  17.0000  17.0000
299   Default  1545.000000      chinese  1545.000000  16.0000  16.0000
207   Default  1255.000000      beijing  1255.000000  15.0000  15.0000
1071  Default  1772.000000       profit  1772.000000  14.0000  14.0000
639   Default   566.000000         gold   566.000000  13.0000  13.0000
636   Default   564.000000           gm   564.000000  12.0000  12.0000
230   Default   552.000000          bre   552.000000  11.0000  11.0000
351   Default   971.000000     computer   971.000000  10.0000  10.0000
437   Default   686.000000         deng   686.000000   9.0000   9.0000
215   Default  3218.000000      billion  3218.000000   8.0000   8.0000
22    Default  1852.000000         1996  1852.000000   7.0000   7.0000
1113  Default  1059.000000         rate  1059.000000   6.0000   6.0000
151   Default   825.000000        apple   825.000000   5.0000   5.0000
1204  Default  2187.000000         sale  2187.000000   4.0000   4.0000
550   Default   770.000000       export   770.000000   3.0000   3.0000
141   Default  3773.000000      analyst  3773.000000   2.0000   2.0000
259   Default   494.000000       canada   494.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
717   Topic10   124.910553    indonesia   160.094879   2.9664  -5.0811
1347  Topic10    78.522644    surcharge   101.793964   2.9550  -5.5453
198   Topic10    85.783665   bankruptcy   121.326701   2.8679  -5.4569
930   Topic10   110.906275      norfolk   184.596623   2.7051  -5.2000
441   Topic10   109.773620      deposit   262.652988   2.3421  -5.2103
1234  Topic10    45.285984     services   120.239830   2.2380  -6.0957
377   Topic10    88.764629       copper   245.071807   2.1990  -5.4227
675   Topic10    34.991859         hire    97.318556   2.1917  -6.3536
213   Topic10    35.683009      bidding   102.930756   2.1552  -6.3340
1180  Topic10    50.230209         rich   145.301418   2.1523  -5.9921
1162  Topic10    40.473791     resource   119.534584   2.1316  -6.2080
368   Topic10    31.723026   consultant    94.240875   2.1257  -6.4516
1076  Topic10   142.467909      project   449.634017   2.0652  -4.9496
1315  Topic10   572.894232        stock  22

## <span style="text-decoration: underline;">Neural Networks</span><a id='nn_ml'></a> [(to top)](#toc)

Interested? Check out the Stanford course CS224n ([Syllabus](http://web.stanford.edu/class/cs224n/syllabus.html))!   